In [1]:
import sys
sys.path.append("..")
from allthethings import PyNetwork, PyPipe_ps
from allthethings import PyBC_opt_dh
import numpy as np
import matplotlib.pyplot as plt
%pylab inline
from scipy import optimize
from scipy import interpolate
from writeit import rewritePipes

Populating the interactive namespace from numpy and matplotlib


In [11]:
btype = 0   #specifying A (btype =0) or 'Q (btype = 0) at boundary?
case = 0## should be 0,1, or 2

D = .1            #pipe diameter (m)
Mr = 0.007        #manning roughness coeffs
T = 24           #simulation time (s)
a = 100            #pressure wavespeed
Ltot = 500
Np = 10
Nn = Np+1
Ndof = 2
    
L = Ltot/float(Np)
N = 2*int(L)
dx = L/float(N)
M = max(int(T*a/(dx*.8))*5,1)
Qin =0.0087
Ain =  0.00793018096379

if btype==0:
    bval = Ain
else:
    bval = Qin
oldinp = "../indata/DFDpleasework8.inp"
fn = "../indata/optim_elev_test"
params= (D,N,L,Mr,T,M,a,bval,Qin,Np,fn,oldinp,case,btype)
print M

30000


In [24]:
def f(z,*params):
    D,N,L,Mr,T,M,a,bval,Qin,Np,fn,oldinp,case,btype = params
    K = 4
    e0,e1 = z
    y = [25,e0,0,e1,25]
    x = np.linspace(-1,1,K+1)
    t = np.linspace(-1,1,Np+1)
    s = interpolate.splrep(x,y,s=0)
    elevs = interpolate.splev(t,s,der=0)
    print elevs
    if btype ==0:
        h0 =0
    else:
        h0 = 0.00328335033016
    Nn = len(elevs)
    dt = T/float(M)
    Ns = [N]*Np
    Ls = [L]*Np
    Mrs = [Mr]*Np  #manning roughness coeffs
    Ds = [D]*Np      #pipe diameter (m) 
    h0s = [h0]*Np    #IC for h
    q0s = [0]*Np    #IC for Q
    jt = [1]+[2]*(Nn-2)+[1]  #for DFDpleasework2.inp
    bt = [1]*Nn
    bt[0] = btype
    bv = [0]+[1]*(Nn-1)
    r =  [0]+[1]*(Nn-2)+[-1]
    
    (fi, fc) = rewritePipes(fn,oldinp, Ns, Ls, Mrs, Ds, jt, bt, bv, r, h0s, q0s, T, M, a, elevs)
    n1 = PyNetwork(fi,fc,1)

    b00 = bval*np.ones(M+1)
    n1.setbVal(0,b00)
    if btype==0:
        A00 = np.ones(N)*bval
        Q00 = np.ones(N)*Qin
        n1.setIC(0,Q00,A00)
    n1.runForwardProblem(dt)
    H = sum([n1.getAveGradH(i) for i in range(M+1)])
    #print "elevs = [10,%f, %f, %f, 6, %f, %f, 10] and H = %f"%(e0,e1,e2,e3,e4,H)
    return H

In [25]:
z0 = np.random.rand(2)*10
#z0 = (7,7,8,8)
#z0 = (7,7,7,8,8,8)

print z0
f(z0,*params)

[ 5.44468685  0.41194765]
[  2.50000000e+01   1.45311687e+01   7.75018394e+00   3.70361481e+00
   1.43803045e+00  -6.66133815e-16  -1.13873202e+00  -8.05719510e-01
   2.59665901e+00   1.06660250e+01   2.50000000e+01]


375176.21091974399

In [26]:
if case==0:
    cons = ({'type': 'ineq', 'fun': lambda x:  x[0] +10},
        {'type': 'ineq', 'fun': lambda x: -x[0] +13},
        {'type': 'ineq', 'fun': lambda x:  x[1] +10},
        {'type': 'ineq', 'fun': lambda x: -x[1] +13},
       )
elif case==1:
    cons = ({'type': 'ineq', 'fun': lambda x:  x[0] -4},
        {'type': 'ineq', 'fun': lambda x: -x[0] +12},
        {'type': 'ineq', 'fun': lambda x:  x[1] -4},
        {'type': 'ineq', 'fun': lambda x: -x[1] +12},
        {'type': 'ineq', 'fun': lambda x:  x[2] -4},
        {'type': 'ineq', 'fun': lambda x: -x[2] +12},
        {'type': 'ineq', 'fun': lambda x:  x[3] -4},
        {'type': 'ineq', 'fun': lambda x: -x[3] +12},
       )
elif case==2:
    cons = ({'type': 'ineq', 'fun': lambda x:  x[0] -4},
        {'type': 'ineq', 'fun': lambda x: -x[0] +12},
        {'type': 'ineq', 'fun': lambda x:  x[1] -4},
        {'type': 'ineq', 'fun': lambda x: -x[1] +12},
        {'type': 'ineq', 'fun': lambda x:  x[2] -4},
        {'type': 'ineq', 'fun': lambda x: -x[2] +12},
        {'type': 'ineq', 'fun': lambda x:  x[3] -4},
        {'type': 'ineq', 'fun': lambda x: -x[3] +12},
        {'type': 'ineq', 'fun': lambda x:  x[4] -4},
        {'type': 'ineq', 'fun': lambda x: -x[4] +12},
        {'type': 'ineq', 'fun': lambda x:  x[5] -4},
        {'type': 'ineq', 'fun': lambda x: -x[5] +12},
       )

In [27]:
class MyBounds(object):
    def __init__(self, xmax=[13]*Ndof, xmin=[-10]*Ndof):
        self.xmax = np.array(xmax) 
        self.xmin = np.array(xmin)
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))
        return tmax and tmin

In [28]:
def print_f(x,f,accepted):
    print "f = %f x = %s, accepted = %d"%(f,str(x), int(accepted))

In [29]:
%time
minimizer_kwargs = {"method": "COBYLA","constraints":cons,"args":params}
np.random.seed(555)
mybounds = MyBounds()
ret = optimize.basinhopping(f, z0,  minimizer_kwargs=minimizer_kwargs,
                   niter=5, accept_test = mybounds, callback=print_f)

#res = optimize.anneal(f, z, args=params, schedule='boltzmann',
#                          full_output=True, maxiter=50, lower=20,
#                          upper=250, dwell=50, disp=True, feps = 1e-3)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 16.9 µs
[  2.50000000e+01   1.45311687e+01   7.75018394e+00   3.70361481e+00
   1.43803045e+00  -6.66133815e-16  -1.13873202e+00  -8.05719510e-01
   2.59665901e+00   1.06660250e+01   2.50000000e+01]
[  2.50000000e+01   1.54271687e+01   8.83818394e+00   4.53561481e+00
   1.82203045e+00  -6.66133815e-16  -1.26673202e+00  -8.69719510e-01
   2.66065901e+00   1.07940250e+01   2.50000000e+01]
[  2.50000000e+01   1.55551687e+01   8.90218394e+00   4.47161481e+00
   1.69403045e+00  -6.66133815e-16  -8.82732018e-01  -3.77195099e-02
   3.74865901e+00   1.16900250e+01   2.50000000e+01]
[  2.50000000e+01   1.64580423e+01   9.99197204e+00   5.29688066e+00
   2.06785952e+00  -1.44328993e-15  -9.82857128e-01  -4.18734776e-02
   3.89053874e+00   1.18819673e+01   2.50000000e+01]
[  2.50000000e+01   1.73239978e+01   1.10047898e+01   6.02358287e+00
   2.36158397e+00  -1.22124533e-15  -9.36286479e-01   2.52309592e-01
   4.40904890e+00   1.23771921e+01

In [33]:
ret


                  nfev: 188
 minimization_failures: 0
                   fun: 346211.10383121943
                     x: array([ 7.1512121 ,  2.42915963])
               message: ['requested number of basinhopping iterations completed successfully']
                   nit: 5

In [34]:
f0 = f(z0,*params)
ff = ret.fun
print T

[  2.50000000e+01   1.45311687e+01   7.75018394e+00   3.70361481e+00
   1.43803045e+00  -6.66133815e-16  -1.13873202e+00  -8.05719510e-01
   2.59665901e+00   1.06660250e+01   2.50000000e+01]
24


In [35]:
print "Np = &%d"%Np
print "Nfev = &%d"%ret.nfev
print "z_0= &%s"%(str(list(z0)))
print "z_f =& %s"%(str(list(ret.x)))
print "f_0 =& %f, f_f = %f, f_f/f_0=%f"%(f0,ff,ff/f0)

Np = &10
Nfev = &188
z_0= &[5.4446868518465683, 0.41194765454177884]
z_f =& [7.1512121048436992, 2.4291596329269143]
f_0 =& 375176.210920, f_f = 346211.103831, f_f/f_0=0.922796


In [21]:
print "z_0&= %s"%(str(z0))
print "z_f& = %s"%(str(ret.x))
print "f_0& = %f, f_f = %f, f_f/f_0=%f"%(f0,ff,ff/f0)

z_0&= [ 4.76258557  4.12488984]
z_f& = [ 13.  13.]
f_0& = 956150.478984, f_f = 650503.152077, f_f/f_0=0.680336


In [22]:

Np = &6
Nfev = &340
z_0= &[7.5, 6.5, 7.5, 8.5]
z_f =& [8.7409181741445288, 6.4482813556630605, 7.8603789169278508, 7.8071324922307346]
f_0 =& 625689.371403, f_f = 110665.153922, f_f/f_0=0.176869

SyntaxError: invalid syntax (<ipython-input-22-9684ddbf2250>, line 2)

In [ ]:
ret.x